In [1]:
!pip install swig

!pip install box2d-py

!pip install gymnasium

!pip install "gymnasium[atari,accept-rom-license]"

!pip install "stable-baselines3[extra]"

!pip install "pettingzoo[all]"

!pip install supersuit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 8.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 455, in run
    installed = install_given_reqs(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/__init__.

In [2]:
!pip install "autorom[accept-rom-license]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446667 sha256=6fae8a9a5a0b99e2c63836cf26f15e39eedac862d4b6ada7c1fffd80c34f9577
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [3]:
!AutoROM

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms
	/usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: y
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.10/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/

In [4]:
import supersuit as ss
from pettingzoo.atari import pong_v3
import numpy as np
from stable_baselines3 import DQN
import gymnasium
import torch
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import wandb

In [15]:
#same function as train_part4
def make_env():
    #create env
    env = pong_v3.parallel_env(render_mode=None)

    #preprocessing
    env = ss.color_reduction_v0(env, mode="B")
    env = ss.resize_v1(env, x_size=84, y_size=84) #specifying size(84 x 84)
    env = ss.frame_stack_v1(env, 4) #stacking frames (4)
    env = ss.dtype_v0(env, dtype=np.float32)
    env = ss.normalize_obs_v0(env, env_min=0, env_max=1)

    #as we had multiple errors with Gymnasium, we had to create a custom wrapper as the already existing ones were not working
    #custom wrapper --> Gymnasium compatible
    class GymCompatWrapper(gymnasium.Env):
        def __init__(self, env):
            self.env = env
            self.agents = env.possible_agents
            #specify obs space (4, 84, 84)
            self.observation_space = gymnasium.spaces.Box(
                low=0, high=1,
                shape=(4, 84, 84),
                dtype=np.float32
            )
            self.action_space = env.action_space(self.agents[0])
            self.current_side = 0  #which side we're playing from
            self._current_side = 0

        #reset
        def reset(self, **kwargs):
            obs, _ = self.env.reset(**kwargs)
            #choose which side to play from
            self.current_side = np.random.randint(0, 2)
            self._current_side = self.current_side  #external tracker
            obs_agent = obs[self.agents[self.current_side]]

            #flip observation if playing from second side
            if self.current_side == 1:
                obs_agent = np.flip(obs_agent, axis=1)

            obs_agent = np.transpose(obs_agent, (2, 0, 1))
            return obs_agent, {}

        def step(self, action):
            #second side, flip the action
            if self.current_side == 1:
                #0=STAY, 1=UP, 2=DOWN
                if action == 1:
                    action = 2
                elif action == 2:
                    action = 1

            actions = {
                self.agents[0]: action if self.current_side == 0 else self.action_space.sample(),
                self.agents[1]: action if self.current_side == 1 else self.action_space.sample()
            }

            obs, rewards, terminations, truncations, infos = self.env.step(actions)

            obs_agent = obs[self.agents[self.current_side]]
            reward_agent = rewards[self.agents[self.current_side]]
            terminated = terminations[self.agents[self.current_side]]
            truncated = truncations[self.agents[self.current_side]]

            #flip observation if playing from second side
            if self.current_side == 1:
                obs_agent = np.flip(obs_agent, axis=1)
                reward_agent = -reward_agent  #invert reward for second side

            obs_agent = np.transpose(obs_agent, (2, 0, 1))

            return obs_agent, reward_agent, terminated, truncated, infos

        def render(self):
            return self.env.render()

        def close(self):
            return self.env.close()

        def get_current_side(self):
            return self._current_side

    env = GymCompatWrapper(env)
    return env




In [ ]:
def evaluate_model(model_path, num_episodes=100, render=True):
    #env for eval
    env_render = pong_v3.parallel_env(render_mode="rgb_array")
    wrapped_env = make_env()

    #load model
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")

    model = DQN.load(model_path)
    print(f"Loaded model from {model_path}")

    # Add tracking variables for wins
    wins_side0 = 0
    total_side0 = 0
    wins_side1 = 0
    total_side1 = 0

    episode_rewards_side0 = []
    episode_rewards_side1 = []

    for episode in range(num_episodes):
        current_side = episode % 2

        obs, _ = wrapped_env.reset()
        wrapped_env.current_side = current_side

        env_frames = []
        done = False
        episode_reward = 0

        render_obs = env_render.reset()

        while not done:
            #get action from model
            action = model.predict(obs, deterministic=True)[0]

            #step env
            obs, reward, terminated, truncated, _ = wrapped_env.step(action)
            done = terminated or truncated
            episode_reward += reward

            #capture frame
            render_actions = {agent: action for agent in env_render.agents}
            render_obs, _, _, _, _ = env_render.step(render_actions)
            frame = env_render.render()
            frame_pil = Image.fromarray(frame)
            env_frames.append(frame_pil)

        #episode ends
        final_reward = episode_reward if current_side == 0 else -episode_reward
        result = "WON" if final_reward > 0 else "LOST" if final_reward < 0 else "TIED"

        #track wins for each side
        if current_side == 0:
            total_side0 += 1
            if final_reward > 0:
                wins_side0 += 1
        else:
            total_side1 += 1
            if final_reward > 0:
                wins_side1 += 1

        print(f"Episode {episode + 1}/{num_episodes}")
        print(f"Playing as {'first' if current_side == 0 else 'second'} player")
        print(f"Result: {result}")
        print(f"Reward: {final_reward:.2f}")
        print("-" * 50)

        #save rewards
        if current_side == 0:
            episode_rewards_side0.append(episode_reward)
        else:
            episode_rewards_side1.append(-episode_reward)

    wrapped_env.close()
    env_render.close()

    #save video
    if env_frames:
        #init wandb
        wandb.init(project="pong-dqn", name="model_evaluation_single", resume=True)

        #save best episode
        video_path = os.path.join(os.path.dirname(model_path), "best_episode.mp4")
        save_video(env_frames, video_path)
        wandb.log({"best_episode": wandb.Video(video_path, fps=30, format="mp4")})

        #plot results
        plt.figure(figsize=(12, 6))
        plt.plot(episode_rewards_side0, label="agent")
        plt.title("Evaluation Results")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.legend()

        plot_path = os.path.join(os.path.dirname(model_path), "evaluation_results.png")
        plt.savefig(plot_path)

        #log to wandb
        metrics = {
            "evaluation_plot": wandb.Image(plt),
            "evaluation_metrics": {
                "mean_reward": np.mean(episode_rewards_side0),
                "std_reward": np.std(episode_rewards_side0),
                "min_reward": min(episode_rewards_side0),
                "max_reward": max(episode_rewards_side0)
            }
        }
        wandb.log(metrics)

        plt.close()

        wandb.finish()

    #final statistics
    print("\nFinal Statistics:")
    print(f"First player (Side 0):")
    print(f"Wins: {wins_side0}/{total_side0} ({(wins_side0/total_side0)*100:.2f}%)")
    print(f"Mean reward: {np.mean(episode_rewards_side0):.2f} ± {np.std(episode_rewards_side0):.2f}")

    print(f"\nSecond player (Side 1):")
    print(f"Wins: {wins_side1}/{total_side1} ({(wins_side1/total_side1)*100:.2f}%)")
    print(f"Mean reward: {np.mean(episode_rewards_side1):.2f} ± {np.std(episode_rewards_side1):.2f}")

    #win rates to wandb logging
    if wandb.run is not None:
        wandb.log({
            "win_rate_side0": (wins_side0/total_side0)*100,
            "win_rate_side1": (wins_side1/total_side1)*100,
            "total_win_rate": ((wins_side0 + wins_side1)/(total_side0 + total_side1))*100
        })



In [17]:
def save_video(frames, video_path):
    """Helper function to save frames as video"""
    if frames:
        #size first frame
        frame_size = frames[0].size

        #video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(video_path, fourcc, 30.0, frame_size)

        #write frames to video
        for frame in frames:
            #convert PIL image to OpenCV format
            frame_cv = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
            out.write(frame_cv)

        out.release()
        print(f"Video saved at: {video_path}")


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

wandb.login(key="KEY")

#run ID from wandb
run_id = "so9qx1lb"
model_path = f"/content/models/{run_id}/best_model.zip"

#eval model
evaluate_model(
    model_path=model_path,
    num_episodes=100,
    render=True
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Using device: cuda
Loaded model from /content/models/so9qx1lb/best_model.zip
Episode 1/100
Playing as first player
Result: WON
Reward: 18.00
--------------------------------------------------
Episode 2/100
Playing as second player
Result: LOST
Reward: -6.00
--------------------------------------------------
Episode 3/100
Playing as first player
Result: WON
Reward: 18.00
--------------------------------------------------
Episode 4/100
Playing as second player
Result: WON
Reward: 3.00
--------------------------------------------------
Episode 5/100
Playing as first player
Result: WON
Reward: 6.00
--------------------------------------------------
Episode 6/100
Playing as second player
Result: WON
Reward: 5.00
--------------------------------------------------
Episode 7/100
Playing as first player
Result: WON
Reward: 8.00
--------------------------------------------------
Episode 8/100
Playing as second player
Result: LOST
Reward: -5.00
--------------------------------------------------
E

wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


Video saved at: /content/models/so9qx1lb/best_episode.mp4



Final Statistics:
First player (Side 0):
Wins: 50/50 (100.00%)
Mean reward: 14.52 ± 4.33

Second player (Side 1):
Wins: 25/50 (50.00%)
Mean reward: -0.54 ± 8.35
